In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")

In [4]:
from langchain_community.document_loaders import DirectoryLoader,TextLoader
from langchain.prompts import SemanticSimilarityExampleSelector
from langchain_community.vectorstores import Chroma
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.prompts import SemanticSimilarityExampleSelector

In [5]:
loader = DirectoryLoader('data/examples/', glob="**/*.txt", loader_cls=TextLoader)
docs = loader.load()

In [6]:
### Splitting the document into chunks and creating splits
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200,add_start_index=True)
splits = text_splitter.split_documents(docs)

In [7]:
## Creating Open AI Embeddings
embeddings=OpenAIEmbeddings(api_key=os.environ['OPENAI_API_KEY'])

In [8]:
## Loading the Vector data store
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings(),)

In [9]:
## Creating an example selector
example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=2,
)

In [10]:
# The prompt template will load examples by passing the input do the `select_examples` method
example_selector.select_examples({"input": "Lambda_Python"})

[{'source': 'data\\examples\\AWS_Lambda_Python.txt', 'start_index': 126},
 {'source': 'data\\examples\\AWS_Lambda_Python.txt', 'start_index': 0}]

In [11]:
from langchain.prompts import (
    ChatPromptTemplate,
    FewShotChatMessagePromptTemplate,
)

# Define the few-shot prompt.
few_shot_prompt = FewShotChatMessagePromptTemplate(
    # The input variables select the values to pass to the example_selector
    input_variables=["input"],
    example_selector=example_selector,
    # Define how each example will be formatted.
    # In this case, each example will become 2 messages:
    # 1 human, and 1 AI
    example_prompt=ChatPromptTemplate.from_messages(
        [("human", "{input}"), ("ai", "{output}")]
    ),
)

In [12]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a wondrous wizard of math."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

In [13]:
db = Chroma(persist_directory="./chroma_db", embedding_function=embeddings)

retriever = db.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.3})

In [14]:
retrieved_docs = retriever.invoke("CWhat are the specific dependencies or libraries used in the AWS Lambda function that need to be considered during the conversion to GCP Cloud Function?")
retrieved_docs

[Document(page_content='The below points should be considered while converting AWS Lambda Function \nto GCP Cloud Function and vice versa.  \nEvent Sources:  \n• AWS Lambda:  Supports SQS, SNS, S3, Kafka, CloudWatch Events, \nDynamoDB, Kinesis, and more.', metadata={'page': 0, 'source': 'data\\knowledge_base\\ConvertAWSLambaToGCPCloudFunctionViceVersa.pdf'}),
 Document(page_content='Cloud Functions runtime.  \nCode Libraries and Dependencies:  \n• AWS Lambda:  Leverages AWS SDK for interaction with other AWS services.  \n• GCP Cloud Functions:  Uses GCP client libraries for interacting with GCP \nservices.', metadata={'page': 0, 'source': 'data\\knowledge_base\\ConvertAWSLambaToGCPCloudFunctionViceVersa.pdf'}),
 Document(page_content="• GCP Cloud Functions:  Node.js, Python, Go, Java, PHP, Ruby  \no Conversion:  Check runtime compatibility. If your Lambda function uses \nPowerShell or .NET Core, you'll need to rewrite it in a supported GCP \nCloud Functions runtime.", metadata={'page':

In [15]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

system = """You are a code conversion tool that converts code for a specific service from one cloud platform to another service on a different cloud platform.

You will be asked to convert source cloud service code to target cloud service code.

Given a specific user request to convert the code, write a more generic question that needs to be answered in order to know the specific details needed for conversion. \

If you don't recognize a word or acronym to not try to rewrite it.

Write concise questions."""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
step_back = prompt | llm | StrOutputParser()

In [22]:
question = (
    "Covert the given AWS lambda function written in Python to GCP Cloud Function in Python."
)
result = step_back.invoke({"question": question})
print(result)

What are the specific dependencies or libraries used in the AWS Lambda function that need to be considered during the conversion to GCP Cloud Function?


In [23]:
from langchain.utilities import DuckDuckGoSearchAPIWrapper

search = DuckDuckGoSearchAPIWrapper(max_results=4)

def retriever(query):
    return search.run(query)

######################################

retriever(result)

'Click Create. Step 4: Attach the Layer to Your Lambda Function. Finally, attach the newly created layer to your Lambda function: 1. Open your Lambda function in the AWS console. 2. Scroll to the "Layers" section in the function\'s designer. 3. Click "Add a layer". 1. AWS Lambda layers allow you to reuse code and dependencies across multiple Lambda functions. This helps reduce code duplication, manage dependencies better, and decrease the size of your Lambda ... To use it in a new Lambda function, follow these steps: Go to the AWS Lambda service and click on "Create Function". Provide a name for your function and set the runtime to "Python 3.9 ... A nodejs folder is auto-generated for you. In there you\'ll find an empty package.json file and a node_modules folder. If you want to offload other node_modules you can either: cd into the nodejs folder and add the dependencies into the package.json file, or; move all your existing function\'s node_modules content into the layer\'s node_modul

c:\Users\vivprasad\Desktop\Mine\DataScienceCourse\LangChainPractice\RAG_Langchain\myenv\lib\site-packages\curl_cffi\aio.py:205: UserWarning: Curlm alread closed! quitting from process_data
  warnings.warn("Curlm alread closed! quitting from process_data")
